In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from operator import itemgetter

#!pip install pyldavis

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

C:\Users\utilisateur\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#pip install --upgrade ipykernel

# Preprocessing

In [3]:
df = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [4]:
#df = df.drop('publish_date', axis=1)

In [5]:
df['headline_text'].loc[:5]

0    aba decides against community broadcasting lic...
1       act fire witnesses must be aware of defamation
2       a g calls for infrastructure protection summit
3             air nz staff in aust strike for pay rise
4        air nz strike to affect australian travellers
5                    ambitious olsson wins triple jump
Name: headline_text, dtype: object

## Import des modules nltk

In [6]:
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [7]:
#nltk.download()

In [8]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utilisateur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
stop_words = set(stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

## Tokenizer nos titres afin d'obtenir une liste de mots

In [12]:
words_splitted = []
# for i in df['headline_text'][:10]:
#     words_splitted.append(i.split(' '))

#for line in df['headline_text'][:550000]:
for line in df['headline_text']:
    words_splitted.append(nltk.word_tokenize(line))
    # for i in df['headline_text']:
#     print(i)

In [13]:
final_words = []

for word in words_splitted:
    word_without_sw = []
    for w in word:
        if w not in stop_words and len(w) > 3:
            w = WordNetLemmatizer().lemmatize(w, pos='v').lower()
            w = WordNetLemmatizer().lemmatize(w, pos='n')
            w = WordNetLemmatizer().lemmatize(w, pos='a')
            word_without_sw.append(WordNetLemmatizer().lemmatize(w))
    final_words.append(word_without_sw)
final_words 

[['decide', 'community', 'broadcast', 'licence'],
 ['fire', 'witness', 'must', 'aware', 'defamation'],
 ['call', 'infrastructure', 'protection', 'summit'],
 ['staff', 'aust', 'strike', 'rise'],
 ['strike', 'affect', 'australian', 'traveller'],
 ['ambitious', 'olsson', 'win', 'triple', 'jump'],
 ['antic', 'delight', 'record', 'break', 'barca'],
 ['aussie', 'qualifier', 'stosur', 'waste', 'four', 'memphis', 'match'],
 ['aust', 'address', 'security', 'council', 'iraq'],
 ['australia', 'lock', 'timetable'],
 ['australia', 'contribute', 'million', 'iraq'],
 ['barca', 'take', 'record', 'robson', 'celebrate', 'birthday'],
 ['bathhouse', 'plan', 'move', 'ahead'],
 ['hop', 'launceston', 'cycle', 'championship'],
 ['plan', 'boost', 'paroo', 'water', 'supply'],
 ['blizzard', 'bury', 'unite', 'state', 'bill'],
 ['brigadier', 'dismiss', 'report', 'troop', 'harass'],
 ['british', 'combat', 'troop', 'arrive', 'daily', 'kuwait'],
 ['bryant', 'lead', 'lakers', 'double', 'overtime'],
 ['bushfire', 'vict

In [14]:
#df = pd.DataFrame()

In [15]:
df['sentence'] = final_words

In [16]:
#!pip install gensim
from gensim.corpora import Dictionary


In [17]:
# Create a dictionary representation of the documents.
dictionary = Dictionary(df['sentence'])

# Filter out words that occur less than 2 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

In [18]:
dictionary

In [19]:
corpus = [dictionary.doc2bow(word) for word in df['sentence']]

In [20]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 13373
Number of documents: 1103665


In [21]:
corpus[:5]

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(9, 1), (10, 1), (11, 1), (12, 1)],
 [(13, 1), (14, 1), (15, 1), (16, 1)],
 [(16, 1), (17, 1), (18, 1), (19, 1)]]

## Créer un modèle TF IDF

In [22]:
from gensim import corpora, models

tfidf = models.TfidfModel(corpus)
print(tfidf)

TfidfModel(num_docs=1103665, num_nnz=5028074)


In [23]:
corpus_tfidf = tfidf[corpus]
print(corpus_tfidf[0])

[(0, 0.5991582770887491), (1, 0.38156983624583796), (2, 0.5006327574547617), (3, 0.4947531315985863)]


In [24]:
final_words[0]

['decide', 'community', 'broadcast', 'licence']

In [25]:
from pprint import pprint

for doc in corpus_tfidf: 
    pprint(f"Poids des mots {doc}")
    break

('Poids des mots [(0, 0.5991582770887491), (1, 0.38156983624583796), (2, '
 '0.5006327574547617), (3, 0.4947531315985863)]')


## LDA bag of words

In [26]:
import gensim

In [27]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
lda_model = gensim.models.LdaMulticore(corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2)

In [28]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''

#num_topics (int, optional) – The number of topics to be selected, if -1 - all topics will be in result (ordered by significance).
for idx, topic in lda_model.print_topics(-1):
    
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.028*"south" + 0.023*"government" + 0.021*"hour" + 0.018*"water" + 0.017*"west" + 0.014*"flood" + 0.014*"league" + 0.013*"park" + 0.013*"victoria" + 0.011*"john"


Topic: 1 
Words: 0.041*"australia" + 0.021*"trump" + 0.020*"world" + 0.017*"take" + 0.015*"first" + 0.013*"test" + 0.012*"final" + 0.012*"win" + 0.011*"australian" + 0.010*"year"


Topic: 2 
Words: 0.035*"woman" + 0.033*"charge" + 0.030*"court" + 0.022*"murder" + 0.021*"child" + 0.020*"face" + 0.020*"death" + 0.017*"jail" + 0.015*"accuse" + 0.013*"trial"


Topic: 3 
Words: 0.022*"election" + 0.019*"say" + 0.016*"health" + 0.015*"live" + 0.015*"sydney" + 0.013*"tasmania" + 0.013*"show" + 0.012*"labor" + 0.011*"government" + 0.010*"turnbull"


Topic: 4 
Words: 0.031*"fire" + 0.030*"australian" + 0.024*"kill" + 0.022*"house" + 0.021*"warn" + 0.019*"attack" + 0.018*"crash" + 0.012*"dead" + 0.011*"force" + 0.011*"near"


Topic: 5 
Words: 0.019*"market" + 0.016*"china" + 0.014*"rise" + 0.013*"price" + 0.013*"shar

In [29]:
final_words[959]

['yemen', 'chopper', 'crash', 'kill', 'eight', 'troop']

In [30]:
# test

'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 959
# Our test document is document number 4310

for index, score in sorted(lda_model[corpus[document_num]], key=lambda tup: -1*tup[1]):

    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))




Score: 0.7286007404327393	 
Topic: 0.031*"fire" + 0.030*"australian" + 0.024*"kill" + 0.022*"house" + 0.021*"warn" + 0.019*"attack" + 0.018*"crash" + 0.012*"dead" + 0.011*"force" + 0.011*"near"

Score: 0.15709884464740753	 
Topic: 0.028*"south" + 0.023*"government" + 0.021*"hour" + 0.018*"water" + 0.017*"west" + 0.014*"flood" + 0.014*"league" + 0.013*"park" + 0.013*"victoria" + 0.011*"john"

Score: 0.014287835918366909	 
Topic: 0.027*"find" + 0.026*"interview" + 0.020*"miss" + 0.016*"claim" + 0.015*"police" + 0.014*"driver" + 0.012*"search" + 0.010*"violence" + 0.010*"body" + 0.010*"campaign"

Score: 0.014287509955465794	 
Topic: 0.041*"australia" + 0.021*"trump" + 0.020*"world" + 0.017*"take" + 0.015*"first" + 0.013*"test" + 0.012*"final" + 0.012*"win" + 0.011*"australian" + 0.010*"year"

Score: 0.014287509955465794	 
Topic: 0.035*"woman" + 0.033*"charge" + 0.030*"court" + 0.022*"murder" + 0.021*"child" + 0.020*"face" + 0.020*"death" + 0.017*"jail" + 0.015*"accuse" + 0.013*"trial"

S

## LDA Tf Idf

In [31]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [32]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")



Topic: 0 Word: 0.015*"interview" + 0.015*"murder" + 0.014*"charge" + 0.013*"court" + 0.010*"drum" + 0.009*"assault" + 0.009*"jail" + 0.008*"sentence" + 0.008*"police" + 0.007*"woman"


Topic: 1 Word: 0.009*"kill" + 0.007*"attack" + 0.006*"police" + 0.006*"korea" + 0.006*"september" + 0.006*"suspect" + 0.006*"syria" + 0.006*"arrest" + 0.005*"australian" + 0.005*"june"


Topic: 2 Word: 0.011*"royal" + 0.010*"commission" + 0.007*"friday" + 0.006*"august" + 0.006*"award" + 0.005*"teacher" + 0.005*"hill" + 0.005*"celebrate" + 0.005*"paul" + 0.004*"facebook"


Topic: 3 Word: 0.008*"climate" + 0.007*"monday" + 0.007*"asylum" + 0.007*"thursday" + 0.006*"seeker" + 0.006*"change" + 0.006*"social" + 0.006*"april" + 0.006*"centre" + 0.005*"interest"


Topic: 4 Word: 0.011*"miss" + 0.010*"fire" + 0.010*"south" + 0.010*"weather" + 0.009*"find" + 0.009*"search" + 0.009*"police" + 0.007*"body" + 0.007*"violence" + 0.006*"coast"


Topic: 5 Word: 0.014*"trump" + 0.011*"government" + 0.006*"say" + 0.006*

In [33]:
final_words[959]

['yemen', 'chopper', 'crash', 'kill', 'eight', 'troop']

In [34]:
# test

'''
Check which topic our test document belongs to using the LDA TF IDF model.
'''
document_num = 959
# Our test document is document number 4310

for index, score in sorted(lda_model[corpus_tfidf[document_num]], key=lambda tup: -1*tup[1]):

    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))




Score: 0.6001042127609253	 
Topic: 0.031*"fire" + 0.030*"australian" + 0.024*"kill" + 0.022*"house" + 0.021*"warn" + 0.019*"attack" + 0.018*"crash" + 0.012*"dead" + 0.011*"force" + 0.011*"near"

Score: 0.16406512260437012	 
Topic: 0.028*"south" + 0.023*"government" + 0.021*"hour" + 0.018*"water" + 0.017*"west" + 0.014*"flood" + 0.014*"league" + 0.013*"park" + 0.013*"victoria" + 0.011*"john"

Score: 0.029479237273335457	 
Topic: 0.027*"find" + 0.026*"interview" + 0.020*"miss" + 0.016*"claim" + 0.015*"police" + 0.014*"driver" + 0.012*"search" + 0.010*"violence" + 0.010*"body" + 0.010*"campaign"

Score: 0.029478780925273895	 
Topic: 0.041*"australia" + 0.021*"trump" + 0.020*"world" + 0.017*"take" + 0.015*"first" + 0.013*"test" + 0.012*"final" + 0.012*"win" + 0.011*"australian" + 0.010*"year"

Score: 0.029478780925273895	 
Topic: 0.035*"woman" + 0.033*"charge" + 0.030*"court" + 0.022*"murder" + 0.021*"child" + 0.020*"face" + 0.020*"death" + 0.017*"jail" + 0.015*"accuse" + 0.013*"trial"

S

In [35]:
test="mayor, quit, council, function"

## Test modele

In [36]:
stemmer = SnowballStemmer("english")

### Model tuto (juste verbes)

In [37]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    
    result = []
    
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            # print("###############################")
            # print(lemmatize_stemming(token))
            result.append(lemmatize_stemming(token))
    
    return result

In [38]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):

    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))



Score: 0.5249441862106323	 Topic: 0.018*"country" + 0.018*"coast" + 0.018*"rural" + 0.018*"school" + 0.016*"fund"
Score: 0.27501755952835083	 Topic: 0.035*"police" + 0.029*"queensland" + 0.017*"shoot" + 0.016*"arrest" + 0.012*"guilty"
Score: 0.02501358650624752	 Topic: 0.041*"australia" + 0.021*"trump" + 0.020*"world" + 0.017*"take" + 0.015*"first"
Score: 0.025003522634506226	 Topic: 0.028*"south" + 0.023*"government" + 0.021*"hour" + 0.018*"water" + 0.017*"west"
Score: 0.025003522634506226	 Topic: 0.035*"woman" + 0.033*"charge" + 0.030*"court" + 0.022*"murder" + 0.021*"child"
Score: 0.025003522634506226	 Topic: 0.022*"election" + 0.019*"say" + 0.016*"health" + 0.015*"live" + 0.015*"sydney"
Score: 0.025003522634506226	 Topic: 0.031*"fire" + 0.030*"australian" + 0.024*"kill" + 0.022*"house" + 0.021*"warn"
Score: 0.025003522634506226	 Topic: 0.019*"market" + 0.016*"china" + 0.014*"rise" + 0.013*"price" + 0.013*"share"
Score: 0.025003522634506226	 Topic: 0.027*"find" + 0.026*"interview" +

### Model custom avec verbes noms et adverbes

In [39]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    w = text
    w = WordNetLemmatizer().lemmatize(w, pos='v').lower()
    w = WordNetLemmatizer().lemmatize(w, pos='n')
    w = WordNetLemmatizer().lemmatize(w, pos='a')
    return w

# Tokenize and lemmatize
def preprocess(text):
    
    result = []
    
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    
    return result

In [40]:
unseen_document = "Lightning strikes in India kill 38 people"

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):

    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))



Score: 0.29500114917755127	 Topic: 0.024*"change" + 0.022*"mine" + 0.020*"die" + 0.014*"worker" + 0.014*"close"
Score: 0.22116872668266296	 Topic: 0.031*"fire" + 0.030*"australian" + 0.024*"kill" + 0.022*"house" + 0.021*"warn"
Score: 0.20046748220920563	 Topic: 0.018*"country" + 0.018*"coast" + 0.018*"rural" + 0.018*"school" + 0.016*"fund"
Score: 0.18334341049194336	 Topic: 0.035*"woman" + 0.033*"charge" + 0.030*"court" + 0.022*"murder" + 0.021*"child"
Score: 0.016669899225234985	 Topic: 0.028*"south" + 0.023*"government" + 0.021*"hour" + 0.018*"water" + 0.017*"west"
Score: 0.016669875010848045	 Topic: 0.035*"police" + 0.029*"queensland" + 0.017*"shoot" + 0.016*"arrest" + 0.012*"guilty"
Score: 0.016669873148202896	 Topic: 0.027*"find" + 0.026*"interview" + 0.020*"miss" + 0.016*"claim" + 0.015*"police"
Score: 0.0166698656976223	 Topic: 0.041*"australia" + 0.021*"trump" + 0.020*"world" + 0.017*"take" + 0.015*"first"
Score: 0.01666986383497715	 Topic: 0.022*"election" + 0.019*"say" + 0.01

In [41]:
max_list_topics=[]
for value in corpus_tfidf:
    max_topics=lda_model_tfidf.get_document_topics(value)
    max_list_topics.append(max(max_topics, key=itemgetter(1)))

## Visualisation

In [42]:
import pyLDAvis

In [43]:
pyLDAvis.enable_notebook()

In [44]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
pyLDAvis.display(vis)

## rajout nom topics

In [50]:
df['topic']= max_list_topics

In [51]:
df.head()

,publish_date,headline_text,sentence,topic
0,20030219,aba decides against community broadcasting lic...,"[decide, community, broadcast, licence]","(5, 0.46204594)"
1,20030219,act fire witnesses must be aware of defamation,"[fire, witness, must, aware, defamation]","(0, 0.37415177)"
2,20030219,a g calls for infrastructure protection summit,"[call, infrastructure, protection, summit]","(5, 0.48481894)"
3,20030219,air nz staff in aust strike for pay rise,"[staff, aust, strike, rise]","(2, 0.69918114)"
4,20030219,air nz strike to affect australian travellers,"[strike, affect, australian, traveller]","(8, 0.6944238)"


In [57]:
for i in range(10):
    print(lda_model.print_topic(i, 5))    

0.028*"south" + 0.023*"government" + 0.021*"hour" + 0.018*"water" + 0.017*"west"
0.041*"australia" + 0.021*"trump" + 0.020*"world" + 0.017*"take" + 0.015*"first"
0.035*"woman" + 0.033*"charge" + 0.030*"court" + 0.022*"murder" + 0.021*"child"
0.022*"election" + 0.019*"say" + 0.016*"health" + 0.015*"live" + 0.015*"sydney"
0.031*"fire" + 0.030*"australian" + 0.024*"kill" + 0.022*"house" + 0.021*"warn"
0.019*"market" + 0.016*"china" + 0.014*"rise" + 0.013*"price" + 0.013*"share"
0.027*"find" + 0.026*"interview" + 0.020*"miss" + 0.016*"claim" + 0.015*"police"
0.018*"country" + 0.018*"coast" + 0.018*"rural" + 0.018*"school" + 0.016*"fund"
0.035*"police" + 0.029*"queensland" + 0.017*"shoot" + 0.016*"arrest" + 0.012*"guilty"
0.024*"change" + 0.022*"mine" + 0.020*"die" + 0.014*"worker" + 0.014*"close"


In [58]:
dict_topics_name={0:'politics', 1:'world', 2: 'justice',
                  3:'election', 4:'minor news', 5:'economy',
                  6:'interview', 7:'local news', 8:'criminality',9:'society'}

In [62]:
i=0
for topic_id, value in df['topic']:
    df.at[i,'topics_name']=dict_topics_name[topic_id]
    i+=1

In [72]:
a_list = [el[1] for el in df['topic']]

In [75]:
df['topic']=a_list

In [76]:
df.head()

,publish_date,headline_text,sentence,topic,topics_name
0,20030219,aba decides against community broadcasting lic...,"[decide, community, broadcast, licence]",0.462046,economy
1,20030219,act fire witnesses must be aware of defamation,"[fire, witness, must, aware, defamation]",0.374152,politics
2,20030219,a g calls for infrastructure protection summit,"[call, infrastructure, protection, summit]",0.484819,economy
3,20030219,air nz staff in aust strike for pay rise,"[staff, aust, strike, rise]",0.699181,justice
4,20030219,air nz strike to affect australian travellers,"[strike, affect, australian, traveller]",0.694424,criminality


In [84]:
df.to_csv('topics_abc_news2.csv',sep=';',index=False)

In [78]:
import pickle

In [80]:
df.to_pickle('topics_abc_news2')

In [85]:
df.head()

,publish_date,headline_text,sentence,topic,topics_name
0,20030219,aba decides against community broadcasting lic...,"[decide, community, broadcast, licence]",0.462046,economy
1,20030219,act fire witnesses must be aware of defamation,"[fire, witness, must, aware, defamation]",0.374152,politics
2,20030219,a g calls for infrastructure protection summit,"[call, infrastructure, protection, summit]",0.484819,economy
3,20030219,air nz staff in aust strike for pay rise,"[staff, aust, strike, rise]",0.699181,justice
4,20030219,air nz strike to affect australian travellers,"[strike, affect, australian, traveller]",0.694424,criminality
